<a href="https://colab.research.google.com/github/ViniciusCastillo/BootcampAlura_ProjetoModulo2/blob/main/Notebooks/capturaDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Captura de dados da Cobertura de Vacinação
Neste notebook iremos importar, avaliar e preparar uma base de dados de cobertura vacinal do [DataSUS](http://tabnet.datasus.gov.br/cgi/tabcgi.exe?pni/cnv/cpniuf.def). As informações estarão abertar por Unidade da Federação (uf), Imuno e Ano. Além disso inclui a informação de meta de atingimento de cobertura vacinal por imuno, sendo que a mais recente que encontrei é de 2014, na página 6 do relatório do PNI [*Coberturas vacinais no Brasil Período: 2010 - 2014*](https://portalarquivos2.saude.gov.br/images/pdf/2017/agosto/17/AACOBERTURAS-VACINAIS-NO-BRASIL---2010-2014.pdf) que será utilizada na análise exploratória do projeto.

A base final que ficará disponível para as análises se chamará ***dados*** e terá os seguintes campos:

|Campo|Descrição
|:----|:----|
|**uf**|Unidade da federação|
|**sigla**|Sigla da unidade da federação|
|**regiao**|Região do Brasil que a unidade da federação pertence|
|**Imuno**|Imuno da vacina|
|**ano**|Ano da cobertura vacinal (int)|
|**cv**|Cobertura vacinal em % (float)|
|**meta**|Meta de cobertura vacinal em %|
|**atingimento**|relação entre Cobertura vacinal dividido pela meta de cobertura (float)|
|**qualiAtingimento**|Segmentado em 3 níves:|
||- **Meta Atingida**: caso o atingimento seja maior ou igual 1|
||- **Abaixo da Meta**: caso o atingimento seja menor que 1 e maior que 0.5|
||- **Muito Abaixo**: caso o atingimento seja menor que 0.5|

In [7]:
# importando as bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import seaborn as sns
from datetime import date
# configurando formato de apresentação de números
pd.options.display.float_format = "{:,.2f}".format


In [8]:
# testando importar um arquivo para definir as configurações necessárias na função read_csv
# dados = pd.read_csv("https://github.com/ViniciusCastillo/BootcampAlura_ProjetoModulo2/raw/main/Bases/CV_AC.csv", encoding="ISO-8859-1", sep=";", thousands=".", decimal=":", skiprows=4, skipfooter=20)
# display (dados.head())
# dados.tail()

In [9]:
# como os arquivos estão com as siglas das UFs, eu criei um dicionário para facilitar a importação dos dados e nomeação das UFs
ufs_siglas = {"Acre":"AC","Alagoas":"AL","Amapá":"AP","Amazonas":"AM","Bahia":"BA","Ceará":"CE",
           "Distrito Federal":"DF","Espírito Santo":"ES","Goiás":"GO","Maranhão":"MA","Mato Grosso":"MT",
           "Mato Grosso do Sul":"MS","Minas Gerais":"MG","Pará":"PA","Paraíba":"PB","Paraná":"PR",
           "Pernambuco":"PE","Piauí":"PI","Rio de Janeiro":"RJ","Rio Grande do Norte":"RN","Rio Grande do Sul":"RS",
           "Rondônia":"RO","Roraima":"RR","Santa Catarina":"SC","São Paulo":"SP","Sergipe":"SE","Tocantins":"TO"}

# criar dicionário de regiões
ufs_regioes = {"Acre":"Norte","Alagoas":"Nordeste","Amapá":"Norte","Amazonas":"Norte","Bahia":"Nordeste",
               "Ceará":"Nordeste","Distrito Federal":"Centro-Oeste","Espírito Santo":"Sudeste","Goiás":"Centro-Oeste",
               "Maranhão":"Nordeste","Mato Grosso":"Centro-Oeste","Mato Grosso do Sul":"Centro-Oeste",
               "Minas Gerais":"Sudeste","Pará":"Norte","Paraíba":"Nordeste","Paraná":"Sul","Pernambuco":"Nordeste",
               "Piauí":"Nordeste","Rio de Janeiro":"Sudeste","Rio Grande do Norte":"Nordeste",
               "Rio Grande do Sul":"Sudeste","Rondônia":"Norte","Roraima":"Norte","Santa Catarina":"Sul",
               "São Paulo":"Sudeste","Sergipe":"Nordeste","Tocantins":"Norte"}

# criação de uma funação para importar todos os arquivos conforme o dicionário de ufs_siglas
def importar(estados):
  for estado, sigla in estados.items():
    yield pd.read_csv(f"https://github.com/ViniciusCastillo/BootcampAlura_ProjetoModulo2/raw/main/Bases/CV_{sigla}.csv", 
                      encoding="ISO-8859-1", sep=";", thousands=".", decimal=",", skiprows=4, skipfooter=20)

# importação dos dados
dados = pd.concat(importar(ufs_siglas), keys=ufs_siglas.keys()) # chama a função para importar todos os arquivos em um únicos DataFrame
dados[dados.columns[1:]] = dados[dados.columns[1:]].fillna(0).replace("-",0).astype("float64") # zerando as informações faltantes no campos numéricos
dados.index = dados.index.droplevel(1) # exclui o index que não é a uf, para que o DataFrame tenha um único index
dados["uf"] = dados.index # copia o index para o campo uf
dados = dados.melt(id_vars=["uf","Imuno"], value_vars=dados.columns[1:-2], var_name="ano", value_name="cv") # transforma os anos em uma variável
dados["Imuno"] = dados["Imuno"].str[4:] # retira os códigos do Imunos para que vire texto
dados["ano"] = dados["ano"].astype("int64") # transforma os anos em um campo inteiro
dados["sigla"] = dados["uf"].map(ufs_siglas) # inclui o campo siglas
dados["regiao"] = dados["uf"].map(ufs_regioes) # inclui o campo regioes
dados.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


,uf,Imuno,ano,cv,sigla,regiao
0,Acre,BCG,1994,61.01,AC,Norte
1,Acre,Hepatite B em crianças até 30 dias,1994,0.00,AC,Norte
2,Acre,Rotavírus Humano,1994,0.00,AC,Norte
3,Acre,Meningococo C,1994,0.00,AC,Norte
4,Acre,Hepatite B,1994,11.21,AC,Norte


In [10]:
# cria um DataFrame chamado metas com base no pdf do PNI com o Imuno e qual a meta de cobertura vacinal
metas = pd.DataFrame([["BCG",90],["hepatite B",95],["Penta",95],["Poliomielite",95],
                      ["Pneumo 10",95],["VORH",90],["Meningo C",95],["FA",100],
                      ["Tríplice viral",95],["dT",100],["Tetra viral",95],["HPV",80],
                      ["Hepatite A",95],["dTpa",100],["Influenza",80]], 
                     columns=["vacina","meta"]).set_index("vacina")

In [11]:
# cria um DataFrame com o DE PARA do Imuno que pegamos da base do DataSUS para a lista de metas encontrada no pdf do PNI
dePara = pd.DataFrame([["BCG","BCG"],["Hepatite B  em crianças até 30 dias","hepatite B"],
                       ["Rotavírus Humano","VORH"],["Meningococo C","Meningo C"],
                       ["Hepatite B","hepatite B"],["Penta","Penta"],["Pneumocócica","Pneumo 10"],
                       ["Poliomielite","Poliomielite"],["Poliomielite 4 anos","Poliomielite"],
                       ["Febre Amarela","FA"],["Hepatite A","Hepatite A"],
                       ["Pneumocócica(1º ref)","Pneumo 10"],["Meningococo C (1º ref)","Meningo C"],
                       ["Poliomielite(1º ref)","Poliomielite"],["Tríplice Viral  D1","Tríplice viral"],
                       ["Tríplice Viral  D2","Tríplice viral"],["Tetra Viral(SRC+VZ)","Tetra viral"],
                       ["DTP","dT"],["DTP REF (4 e 6 anos)","dT"],["Tríplice Bacteriana(DTP)(1º ref)","dT"],
                       ["Sarampo","Tríplice viral"],["Haemophilus influenzae b","Influenza"],
                       ["Dupla adulto e tríplice acelular gestante","dTpa"],["dTpa gestante","dTpa"]]).set_index(0)

In [12]:
# junta os dois DataFrames acima para ter a meta por Imuno no formato do arquivo do DataSUS e inclui na base de dados
metas2 = dePara.join(metas, on=1).drop(columns=1) # junta os dois DataFrames remevendo os nomes que não são necessários
dados = dados.join(metas2, on="Imuno") # adiciona a informação de meta na base de dados
dados.head()

,uf,Imuno,ano,cv,sigla,regiao,meta
0,Acre,BCG,1994,61.01,AC,Norte,90
1,Acre,Hepatite B em crianças até 30 dias,1994,0.00,AC,Norte,95
2,Acre,Rotavírus Humano,1994,0.00,AC,Norte,90
3,Acre,Meningococo C,1994,0.00,AC,Norte,95
4,Acre,Hepatite B,1994,11.21,AC,Norte,95


In [13]:
# cria função para criar o campo qualitativo do atingimento
def qualitativo(atingimento):
  if atingimento >= 1:
    return " Meta Atingida"
  else:
    if atingimento < 0.5:
      return "Muito Abaixo"
    else:
      if np.isnan(atingimento):
        return np.nan
      else:
        return "Abaixo da Meta"

dados["atingimento"] = dados["cv"]/dados["meta"] # cria o campo atingimento
dados["qualiAtingimento"] = dados["atingimento"].map(qualitativo) # cria o campo qualiAtingimento
display (dados.head())
dados.tail()

,uf,Imuno,ano,cv,sigla,regiao,meta,atingimento,qualiAtingimento
0,Acre,BCG,1994,61.01,AC,Norte,90,0.68,Abaixo da Meta
1,Acre,Hepatite B em crianças até 30 dias,1994,0.00,AC,Norte,95,0.00,Muito Abaixo
2,Acre,Rotavírus Humano,1994,0.00,AC,Norte,90,0.00,Muito Abaixo
3,Acre,Meningococo C,1994,0.00,AC,Norte,95,0.00,Muito Abaixo
4,Acre,Hepatite B,1994,11.21,AC,Norte,95,0.12,Muito Abaixo


,uf,Imuno,ano,cv,sigla,regiao,meta,atingimento,qualiAtingimento
16843,Tocantins,Tríplice Bacteriana(DTP)(1º ref),2019,47.30,TO,Norte,100,0.47,Muito Abaixo
16844,Tocantins,Sarampo,2019,0.00,TO,Norte,95,0.00,Muito Abaixo
16845,Tocantins,Haemophilus influenzae b,2019,0.00,TO,Norte,80,0.00,Muito Abaixo
16846,Tocantins,Dupla adulto e tríplice acelular gestante,2019,36.77,TO,Norte,100,0.37,Muito Abaixo
16847,Tocantins,dTpa gestante,2019,53.80,TO,Norte,100,0.54,Abaixo da Meta


In [23]:
# código utilizado para avaliar e retirar dados que não seriam úteis para a análise
# corte de anos levou em conta que o ano de 2019 não está completo, no TabNet do DataSUS diz que a coleta foi até 04/09/2019
# também que as informação de 2013 tem algums pontos de atenção e para trás não tinha o mesmo padrão de informação

# lista de imunos que não tem informações boas no período a ser analisado ou são muito parecidos com algum outro que foi mantido
excluirAnalise = ["DTP REF (4 e 6 anos)","Haemophilus influenzae b","Poliomielite 4 anos","Sarampo",
                  "Hepatite B  em crianças até 30 dias","Meningococo C (1º ref)",
                  "Pneumocócica(1º ref)","Poliomielite(1º ref)","dTpa gestante",
                  "Dupla adulto e tríplice acelular gestante","Tríplice Bacteriana(DTP)(1º ref)"]
dados = dados.query("ano > 2013 and ano < 2019 and Imuno not in @excluirAnalise") # retirando os dados que não farão parte da análise
#criando um tabela para visualização da qualidade dos números da seleção
valores = pd.crosstab(index=[dados["Imuno"]],columns=dados["ano"], values = dados["atingimento"], aggfunc = {'mean', 'median'}) * 100 
valores

mean                       ... median                     
ano                   2014   2015   2016   2017  ...   2015   2016   2017   2018
Imuno                                            ...                            
BCG                 120.99 117.82 109.52 107.80  ... 117.56 107.90 107.97 111.80
DTP                  94.11  95.03  90.27  82.64  ...  95.43  88.94  83.73  88.84
Febre Amarela        55.34  53.61  53.45  52.04  ...  66.67  67.58  62.74  68.28
Hepatite A           59.61  99.74  78.09  87.07  ... 101.32  78.89  89.08  86.79
Hepatite B           99.55 100.86 108.98  89.03  ... 102.38 110.20  91.07  97.27
Meningococo C        99.99 101.90  97.96  90.22  ... 100.60  97.29  91.98  92.49
Penta                98.51  99.65  94.85  86.84  ...  99.86  93.14  88.04  93.44
Pneumocócica         95.59  96.24 101.30  95.11  ...  96.89 100.23  95.82  98.51
Poliomielite        101.21 102.70  88.85  87.43  ... 102.52  89.23  88.82  93.78
Rotavírus Humano    102.64 104.25  99.46  92.79  ... 104.64 100.36  93.79 101.22
Tetra Viral(SRC+VZ)  92.75  76.89  83.89  49.04  ...  78.52  85.81  57.06  49.57
Tríplice Viral  D1  120.29 100.17 100.87  94.79  ... 100.44 101.11  93.94  95.15
Tríplice Viral  D2   96.44  80.71  81.83  77.61  ...  83.06  82.47  78.08  82.08

[13 rows x 10 columns]

In [25]:
dados["cv"].where(dados["cv"]<100, 100, inplace=True)